<p style="font-weight:bold;"> <span style="font-size: 36px"> Test Aoc Management </span> </p>

In [1]:
#!eval-notebook "../Initialization/InitSystemorphBaseToMemory"

# Workspace Initialization 

In [1]:
Workspace.InitializeFrom(DataSource);

# Test

In [1]:
var args = new ImportArgs("CH", 2021, 3, Periodicity.Quarterly, null, ImportFormats.Cashflow);
var partition = Workspace.Query<PartitionByReportingNodeAndPeriod>().FirstOrDefault(x => x.ReportingNode == args.ReportingNode && x.Year == args.Year && 
                                                                                    x.Month == args.Month && x.Scenario == args.Scenario);
if(partition == null) ApplicationMessage.Log(Error.PartitionNotFound);

In [1]:
public async Task CheckAocStepStructureAsync(IEnumerable<BaseDataRecord> inputVariables, 
                                             Dictionary<AocStep,IEnumerable<AocStep>> parentBm, 
                                             Dictionary<AocStep,AocStep> referenceBm, 
                                             Dictionary<AocStep,IEnumerable<AocStep>> fullAocBm,
                                             Dictionary<AocStep,IEnumerable<AocStep>> parentBmCdr = null)
{
    //Save test input data
    var importFormat = ImportFormats.Cashflow;
    var inputSource = InputSource.Cashflow;
    if (inputVariables.First() is RawVariable)
    {
        await Workspace.UpdateAsync<RawVariable>(inputVariables.Cast<RawVariable>());
    }
    if (inputVariables.First() is IfrsVariable)
    {
        await Workspace.UpdateAsync<IfrsVariable>(inputVariables.Cast<IfrsVariable>());
        importFormat = ImportFormats.Actual;
        inputSource = InputSource.Actual;
    }
    
    var newArgs = args with {ImportFormat = importFormat };
    //Set up import storage and test universe
    var testStorage = new ImportStorage(newArgs, DataSource, Workspace);
    await testStorage.InitializeAsync();
    var isReinsurance = testStorage.DataNodeDataBySystemName[inputVariables.First().DataNode].IsReinsurance;
    var testUniverse = Scopes.ForStorage(testStorage).ToScope<IModel>();
    //Clean up Workspace
    await Workspace.DeleteAsync<RawVariable>(await Workspace.Query<RawVariable>().ToArrayAsync());   
    await Workspace.DeleteAsync<IfrsVariable>(await Workspace.Query<IfrsVariable>().ToArrayAsync());   
    
    var errors = new List<string>();
    
    var goc = inputVariables.First().DataNode;
    var identities = testUniverse.GetScopes<GetIdentities>(testStorage.DataNodesByImportScope[ImportScope.Primary].Where(dn => dn == goc)).SelectMany(s => s.Identities);
    
   //Assert Parents
    if (importFormat != ImportFormats.Actual)
    {
        var parents = testUniverse.GetScopes<ParentAocStep>(identities.Select(id => (object)(id, "PR")), o => o.WithStorage(testStorage)).Where(x => x.Values != Enumerable.Empty<AocStep>()).ToArray();
        if (parentBm.Count() != parents.Count()) 
        {
            var computedIds = parents.Select(s => $"AocType:{s.Identity.Id.AocType}, Novelty:{s.Identity.Id.Novelty}");
            var expectedIds = parentBm.Keys.Select(aoc => $"AocType:{aoc.AocType}, Novelty:{aoc.Novelty}");
            errors.Add( $"Parent count does not match expected: \n Computed {parents.Count()} \n {string.Join("\n", computedIds)} \n Expected {parentBm.Count()} \n {string.Join("\n", expectedIds)}." );
        }
    
    foreach(var kvp in parentBm)
    {
        var scopeParents = parents.Where(y => y.Identity.Id.AocType == kvp.Key.AocType && y.Identity.Id.Novelty == kvp.Key.Novelty);
        if(scopeParents.Count() != 1)
            errors.Add( $"Parent not found for AocStep: {kvp.Key.AocType}, {kvp.Key.Novelty}.");
        else{
            var scopeParent = scopeParents.First();
            if( kvp.Value.Intersect(scopeParent.Values).Count() != kvp.Value.Count() ||
                kvp.Value.Intersect(scopeParent.Values).Count() != scopeParent.Values.Count()){
                var computedAocSteps = scopeParent.Values.Select(aoc => $"AocType:{aoc.AocType}, Novelty:{aoc.Novelty}");
                var expectedAocSteps = kvp.Value.Select(aoc => $"AocType:{aoc.AocType}, Novelty:{aoc.Novelty}"); 
                errors.Add( $"Parents of {kvp.Key.AocType}, {kvp.Key.Novelty} do not match expected value: \n Computed: \n {string.Join("\n", computedAocSteps)} \n Expected: \n {string.Join("\n", expectedAocSteps)}." );
            }
        }
    }
    
    //Parents for CDR
    if(isReinsurance)
    {
        var parentsCdr = testUniverse.GetScopes<ParentAocStep>(identities.Select(id => (object)(id, AmountTypes.CDR)), o => o.WithStorage(testStorage));
        
        var countP = parentsCdr.Where(x => x.Values != Enumerable.Empty<AocStep>()).Count();
        if (parentBmCdr.Count() != countP) 
            errors.Add( $"Parent count for AmountType CDR does not match expected: \n Computed {countP} \n Expected {parentBm.Count()}." );
    
        foreach(var kvp in parentBmCdr)
        {
            var scopeParents = parentsCdr.Where(y => y.Identity.Id.AocType == kvp.Key.AocType && y.Identity.Id.Novelty == kvp.Key.Novelty);
            if(scopeParents.Count() != 1)
                errors.Add( $"Parent for CDR not found for AocStep: {kvp.Key.AocType}, {kvp.Key.Novelty}.");
            else{
                var scopeParent = scopeParents.First();
                if( kvp.Value.Intersect(scopeParent.Values).Count() != kvp.Value.Count() || 
                    kvp.Value.Intersect(scopeParent.Values).Count() != scopeParent.Values.Count() ){
                    var computedAocSteps = scopeParent.Values.Select(aoc => $"AocType:{aoc.AocType}, Novelty:{aoc.Novelty}");
                    var expectedAocSteps = kvp.Value.Select(aoc => $"AocType:{aoc.AocType}, Novelty:{aoc.Novelty}"); 
                    errors.Add( $"Parents of {kvp.Key.AocType}, {kvp.Key.Novelty} for AmountType CDR do not match expected value: \n Computed: \n {string.Join("\n", computedAocSteps)} \n Expected: \n {string.Join("\n", expectedAocSteps)}." );
                }
            }
        } 
    }
    }
    
    //Assert Reference
    if (importFormat != ImportFormats.Actual)
    {
        var reference = testUniverse.GetScopes<ReferenceAocStep>(identities, o => o.WithStorage(testStorage)).ToArray();
        var countR = reference.Select(x => x.Value).Count();
        if (referenceBm.Count() != countR) 
            errors.Add( $"Reference count does not match expected: \n Computed {countR} \n Expected {referenceBm.Count()}." );
    
        foreach(var kvp in referenceBm)
        {
            var scopeReferences = reference.Where(y => y.Identity.AocType == kvp.Key.AocType && y.Identity.Novelty == kvp.Key.Novelty);
            if(scopeReferences.Count() != 1)
                errors.Add( $"Reference not found for AocStep: {kvp.Key.AocType}, {kvp.Key.Novelty}.");
            else{
                var scopeReference = scopeReferences.First();
                if( kvp.Value.AocType != scopeReference.Value.AocType || kvp.Value.Novelty != scopeReference.Value.Novelty )
                    errors.Add( $"Reference of {kvp.Key.AocType}, {kvp.Key.Novelty} do not match expected value: \n Computed {scopeReference.Value} \n Expected {kvp.Value}." );
            }
        }
    }
    
    //Assert FullAoc
    var fullAoc = testUniverse.GetScopes<PreviousAocSteps>(identities.Select(id => (object)(id, inputSource)), o => o.WithStorage(testStorage)).Where(s => s.Values.Any());
    var count = fullAoc.Count();
    if (fullAocBm.Count() != count)
    {
        var computedAocSteps = fullAoc.Select(x => $"AocType:{x.Identity.Id.AocType}, Novelty:{x.Identity.Id.Novelty}");
        var benchmarkKeys = fullAocBm.Keys.Select(aoc => $"AocType:{aoc.AocType}, Novelty:{aoc.Novelty}");
        errors.Add( $"Full Aoc count does not match expected: \n Computed {count} \n Expected {fullAocBm.Count()}." );    
        errors.Add( $"In particular, \n Computed Identities \n {string.Join("\n", computedAocSteps)} \n Expected \n {string.Join("\n", benchmarkKeys)}." );    
    }
 
    foreach(var kvp in fullAocBm)
    {
       var scopeAocFulls = fullAoc.Where(y => y.Identity.Id.AocType == kvp.Key.AocType && y.Identity.Id.Novelty == kvp.Key.Novelty);
        if(scopeAocFulls.Count() != 1){
            errors.Add( $"Full AocStep not found for AocStep: {kvp.Key.AocType}, {kvp.Key.Novelty}.");}
        else{
            var scopeAocFull = scopeAocFulls.First();
            if( kvp.Value.Intersect(scopeAocFull.Values).Count() != kvp.Value.Count() || 
                kvp.Value.Intersect(scopeAocFull.Values).Count() != scopeAocFull.Values.Count() ){
                var computedAocSteps = scopeAocFull.Values.Select(aoc => $"AocType:{aoc.AocType}, Novelty:{aoc.Novelty}");
                var expectedAocSteps = kvp.Value.Select(aoc => $"AocType:{aoc.AocType}, Novelty:{aoc.Novelty}"); 
                errors.Add( $"AocFull for {kvp.Key.AocType}, {kvp.Key.Novelty} do not match expected value: \n Computed \n {string.Join("\n", computedAocSteps)} \n Expected: \n {string.Join("\n", expectedAocSteps)}." );
            }
        }
    }
    
    if(errors.Any()) ApplicationMessage.Log(Error.Generic, string.Join("\n", errors));
}


# Use Cases

## Gross Cashflow: InForce, NewBusiness, and Combined

In [1]:
var groupOfContract = "DT1.1";

In [1]:
var inputRawVariables = new RawVariable[]{
    new RawVariable{Partition = partition.Id, Values = new []{1.0}, DataNode = groupOfContract, AocType = "BOP", Novelty = "I", AccidentYear = null, AmountType = "PR", CalculationType = "BE"},
    new RawVariable{Partition = partition.Id, Values = new []{1.0}, DataNode = groupOfContract, AocType = "MC", Novelty = "I", AccidentYear = null, AmountType = "PR", CalculationType = "BE"},
    new RawVariable{Partition = partition.Id, Values = new []{1.0}, DataNode = groupOfContract, AocType = "BOP", Novelty = "N", AccidentYear = null, AmountType = "PR", CalculationType = "BE"},
    new RawVariable{Partition = partition.Id, Values = new []{1.0}, DataNode = groupOfContract, AocType = "EV", Novelty = "N", AccidentYear = null, AmountType = "PR", CalculationType = "BE"},
    new RawVariable{Partition = partition.Id, Values = new []{1.0}, DataNode = groupOfContract, AocType = "CL", Novelty = "C", AccidentYear = null, AmountType = "PR", CalculationType = "BE"},
};

In [1]:
var parentBm = new Dictionary<AocStep,IEnumerable<AocStep>>()
    {
        {new AocStep("MC","I"), new AocStep[]{new AocStep("BOP","I")}},
        {new AocStep("YCU","I"), new AocStep[]{new AocStep("MC","I")}}, 
        {new AocStep("EV","N"), new AocStep[]{new AocStep("BOP","N")}},
        {new AocStep("CL","C"), new AocStep[]{new AocStep("YCU","I"), new AocStep("EV","N"),}},
    };

In [1]:
var referenceBm = new Dictionary<AocStep,AocStep>()
    {
        {new AocStep("BOP","I"), new AocStep("BOP","I")},
        {new AocStep("MC","I"), new AocStep("MC","I")},
        {new AocStep("CF","I"), new AocStep("MC","I")},
        {new AocStep("IA","I"), new AocStep("MC","I")},
        {new AocStep("YCU","I"), new AocStep("MC","I")}, 
        {new AocStep("BOP","N"), new AocStep("BOP","N")},
        {new AocStep("CF","N"), new AocStep("BOP","N")},
        {new AocStep("IA","N"), new AocStep("BOP","N")},
        {new AocStep("EV","N"), new AocStep("EV","N")},
        {new AocStep("CL","C"), new AocStep("CL","C")},
        {new AocStep("EA","C"), new AocStep("CF","C")},
        {new AocStep("AM","C"), new AocStep("CL","C")},
        {new AocStep("EOP","C"), new AocStep("CL","C")},
        {new AocStep("CF","C"), new AocStep("CF","C")},
    };

In [1]:
var fullAocBm = new Dictionary<AocStep,IEnumerable<AocStep>>()
    {
        //{new AocStep("BOP","I"), Enumerable.Empty<AocStep>()},
        {new AocStep("MC","I"), new AocStep[]{new AocStep("BOP","I")}},
        {new AocStep("CF","I"), new AocStep[]{new AocStep("BOP","I"), new AocStep("MC","I")}},
        {new AocStep("IA","I"), new AocStep[]{new AocStep("BOP","I"), new AocStep("MC","I"), new AocStep("CF","I")}},
        {new AocStep("YCU","I"), new AocStep[]{new AocStep("IA","I"), new AocStep("BOP","I"), new AocStep("MC","I"), new AocStep("CF","I")}},
        //{new AocStep("BOP","N"), Enumerable.Empty<AocStep>()},
        {new AocStep("CF","N"), new AocStep[]{new AocStep("BOP","N")}},
        {new AocStep("IA","N"), new AocStep[]{new AocStep("BOP","N"), new AocStep("CF","N")}},
        {new AocStep("EV","N"), new AocStep[]{new AocStep("BOP","N"), new AocStep("CF","N"), new AocStep("IA","N")}},
        
        {new AocStep("CL","C"), new AocStep[]{new AocStep("BOP","I"), new AocStep("MC","I"), new AocStep("CF","I"), new AocStep("IA","I"), new AocStep("YCU","I"),
                                              new AocStep("BOP","N"), new AocStep("CF","N"), new AocStep("IA","N"), new AocStep("EV","N"),
                                              }},
        {new AocStep("EA","C"), new AocStep[]{new AocStep("BOP","I"), new AocStep("MC","I"),new AocStep("CF","I"), new AocStep("IA","I"),new AocStep("YCU","I"), 
                                              new AocStep("BOP","N"), new AocStep("CF","N"), new AocStep("IA","N"), new AocStep("EV","N"), 
                                              new AocStep("CL","C"),
                                             }},
        {new AocStep("AM","C"), new AocStep[]{new AocStep("BOP","I"), new AocStep("MC","I"), new AocStep("CF","I"), new AocStep("IA","I"), new AocStep("YCU","I"),
                                              new AocStep("BOP","N"), new AocStep("CF","N"), new AocStep("IA","N"), new AocStep("EV","N"), 
                                              new AocStep("EA","C"), new AocStep("CL","C"),
                                             }},
        {new AocStep("EOP","C"), new AocStep[]{new AocStep("BOP","I"), new AocStep("MC","I"), new AocStep("CF","I"), new AocStep("IA","I"), new AocStep("YCU","I"),
                                               new AocStep("BOP","N"), new AocStep("CF","N"), new AocStep("IA","N"), new AocStep("EV","N"), 
                                                new AocStep("EA","C"), new AocStep("AM","C"), new AocStep("CL","C"),
                                               }},
        //{new AocStep("CF","C"), Enumerable.Empty<AocStep>()},
        
    };

In [1]:
await CheckAocStepStructureAsync(inputRawVariables, parentBm, referenceBm, fullAocBm)

## Reinsurance Cashflow: InForce, NewBusiness, and Combined

In [1]:
groupOfContract = "DTR1.1";

In [1]:
inputRawVariables = new RawVariable[]{
    new RawVariable{Partition = partition.Id, Values = new []{1.0}, DataNode = groupOfContract, AocType = "BOP", Novelty = "I", AccidentYear = null, AmountType = "PR", CalculationType = "BE"},
    new RawVariable{Partition = partition.Id, Values = new []{1.0}, DataNode = groupOfContract, AocType = "MC", Novelty = "I", AccidentYear = null, AmountType = "PR", CalculationType = "BE"},
    new RawVariable{Partition = partition.Id, Values = new []{1.0}, DataNode = groupOfContract, AocType = "BOP", Novelty = "N", AccidentYear = null, AmountType = "PR", CalculationType = "BE"},
    new RawVariable{Partition = partition.Id, Values = new []{1.0}, DataNode = groupOfContract, AocType = "EV", Novelty = "N", AccidentYear = null, AmountType = "PR", CalculationType = "BE"},
    new RawVariable{Partition = partition.Id, Values = new []{1.0}, DataNode = groupOfContract, AocType = "CL", Novelty = "C", AccidentYear = null, AmountType = "PR", CalculationType = "BE"},
};

In [1]:
parentBm = new Dictionary<AocStep,IEnumerable<AocStep>>()
    {
        {new AocStep("MC","I"), new AocStep[]{new AocStep("BOP","I")}},
        {new AocStep("YCU","I"), new AocStep[]{new AocStep("MC","I")}},
        {new AocStep("CRU","I"), new AocStep[]{new AocStep("YCU","I")}}, 
        {new AocStep("EV","N"), new AocStep[]{new AocStep("BOP","N")}},
        {new AocStep("CL","C"), new AocStep[]{new AocStep("YCU","I"), new AocStep("EV","N"),}},
    };

In [1]:
var parentBm_CDR = new Dictionary<AocStep,IEnumerable<AocStep>>()
    {
        {new AocStep("MC","I"), new AocStep[]{new AocStep("BOP","I")}},
        {new AocStep("YCU","I"), new AocStep[]{new AocStep("MC","I")}}, 
        {new AocStep("CRU","I"), new AocStep[]{new AocStep("YCU","I")}}, 
        {new AocStep("EV","N"), new AocStep[]{new AocStep("BOP","N")}},
        {new AocStep("CL","C"), new AocStep[]{new AocStep("CRU","I"), new AocStep("EV","N"),}},
    };

In [1]:
referenceBm = new Dictionary<AocStep,AocStep>()
    {
        {new AocStep("BOP","I"), new AocStep("BOP","I")},
        {new AocStep("MC","I"), new AocStep("MC","I")},
        {new AocStep("RCU","I"), new AocStep("MC","I")}, 
        {new AocStep("CF","I"), new AocStep("MC","I")},
        {new AocStep("IA","I"), new AocStep("MC","I")},
        {new AocStep("YCU","I"), new AocStep("MC","I")}, 
        {new AocStep("CRU","I"), new AocStep("MC","I")}, 
        {new AocStep("BOP","N"), new AocStep("BOP","N")},
        {new AocStep("CF","N"), new AocStep("BOP","N")},
        {new AocStep("IA","N"), new AocStep("BOP","N")},
        {new AocStep("EV","N"), new AocStep("EV","N")},
        {new AocStep("CL","C"), new AocStep("CL","C")},
        {new AocStep("EA","C"), new AocStep("CF","C")},
        {new AocStep("AM","C"), new AocStep("CL","C")},
        {new AocStep("EOP","C"), new AocStep("CL","C")},
        {new AocStep("CF","C"), new AocStep("CF","C")},
    };

In [1]:
fullAocBm = new Dictionary<AocStep,IEnumerable<AocStep>>()
    {
        //{new AocStep("BOP","I"), Enumerable.Empty<AocStep>()},
        {new AocStep("MC","I"), new AocStep[]{new AocStep("BOP","I")}},
        {new AocStep("RCU","I"), new AocStep[]{new AocStep("BOP","I"), new AocStep("MC","I")}},
        {new AocStep("CF","I"), new AocStep[]{new AocStep("RCU","I"), new AocStep("BOP","I"), new AocStep("MC","I")}},
        {new AocStep("IA","I"), new AocStep[]{new AocStep("RCU","I"), new AocStep("BOP","I"), new AocStep("MC","I"), new AocStep("CF","I")}},
        {new AocStep("YCU","I"), new AocStep[]{new AocStep("IA","I"), new AocStep("RCU","I"), new AocStep("BOP","I"), new AocStep("MC","I"), new AocStep("CF","I")}},
        {new AocStep("CRU","I"), new AocStep[]{new AocStep("YCU","I"), new AocStep("IA","I"), new AocStep("RCU","I"), new AocStep("BOP","I"), new AocStep("MC","I"), new AocStep("CF","I")}},
        //{new AocStep("BOP","N"), Enumerable.Empty<AocStep>()},
        {new AocStep("CF","N"), new AocStep[]{new AocStep("BOP","N")}},
        {new AocStep("IA","N"), new AocStep[]{new AocStep("BOP","N"), new AocStep("CF","N")}},
        {new AocStep("EV","N"), new AocStep[]{new AocStep("BOP","N"), new AocStep("CF","N"), new AocStep("IA","N")}},
        
        {new AocStep("CL","C"), new AocStep[]{new AocStep("BOP","I"), new AocStep("MC","I"), new AocStep("RCU","I"), new AocStep("CF","I"), new AocStep("IA","I"), new AocStep("YCU","I"), new AocStep("CRU","I"),
                                              new AocStep("BOP","N"), new AocStep("CF","N"), new AocStep("IA","N"), new AocStep("EV","N"),
                                              }},
        {new AocStep("EA","C"), new AocStep[]{new AocStep("BOP","I"), new AocStep("MC","I"), new AocStep("RCU","I"), new AocStep("CF","I"), new AocStep("IA","I"),new AocStep("YCU","I"), new AocStep("CRU","I"),
                                              new AocStep("BOP","N"), new AocStep("CF","N"), new AocStep("IA","N"), new AocStep("EV","N"), 
                                              new AocStep("CL","C"),
                                             }},
        {new AocStep("AM","C"), new AocStep[]{new AocStep("BOP","I"), new AocStep("MC","I"), new AocStep("RCU","I"), new AocStep("CF","I"), new AocStep("IA","I"),new AocStep("YCU","I"), new AocStep("CRU","I"),
                                              new AocStep("BOP","N"), new AocStep("CF","N"), new AocStep("IA","N"), new AocStep("EV","N"), 
                                              new AocStep("EA","C"), new AocStep("CL","C"),
                                             }},
        {new AocStep("EOP","C"), new AocStep[]{new AocStep("BOP","I"), new AocStep("MC","I"), new AocStep("RCU","I"), new AocStep("CF","I"), new AocStep("IA","I"),new AocStep("YCU","I"), new AocStep("CRU","I"),
                                               new AocStep("BOP","N"), new AocStep("CF","N"), new AocStep("IA","N"), new AocStep("EV","N"), 
                                               new AocStep("EA","C"), new AocStep("AM","C"), new AocStep("CL","C"),
                                               }},
        //{new AocStep("CF","C"), Enumerable.Empty<AocStep>()},
        
    };

In [1]:
await CheckAocStepStructureAsync(inputRawVariables, parentBm, referenceBm, fullAocBm, parentBm_CDR)

## Cashflow: InForce, and Combined

In [1]:
var groupOfContract = "DT1.1";

In [1]:
inputRawVariables = new RawVariable[]{
    new RawVariable{Partition = partition.Id, Values = new []{1.0}, DataNode = groupOfContract, AocType = "BOP", Novelty = "I", AccidentYear = null, AmountType = "PR", CalculationType = "BE"},
    new RawVariable{Partition = partition.Id, Values = new []{1.0}, DataNode = groupOfContract, AocType = "MC", Novelty = "I", AccidentYear = null, AmountType = "PR", CalculationType = "BE"},
    new RawVariable{Partition = partition.Id, Values = new []{1.0}, DataNode = groupOfContract, AocType = "CL", Novelty = "C", AccidentYear = null, AmountType = "PR", CalculationType = "BE"},
};

In [1]:
parentBm = new Dictionary<AocStep,IEnumerable<AocStep>>()
    {
        {new AocStep("MC","I"), new AocStep[]{new AocStep("BOP","I")}},
        {new AocStep("YCU","I"), new AocStep[]{new AocStep("MC","I")}}, 
        {new AocStep("CL","C"), new AocStep[]{new AocStep("YCU","I")}},
    };

In [1]:
referenceBm = new Dictionary<AocStep,AocStep>()
    {
        {new AocStep("BOP","I"), new AocStep("BOP","I")},
        {new AocStep("MC","I"), new AocStep("MC","I")},
        {new AocStep("CF","I"), new AocStep("MC","I")},
        {new AocStep("IA","I"), new AocStep("MC","I")},
        {new AocStep("YCU","I"), new AocStep("MC","I")}, 
        {new AocStep("CL","C"), new AocStep("CL","C")},
        {new AocStep("EA","C"), new AocStep("CF","C")},
        {new AocStep("AM","C"), new AocStep("CL","C")},
        {new AocStep("EOP","C"), new AocStep("CL","C")},
        {new AocStep("CF","C"), new AocStep("CF","C")},
    };

In [1]:
fullAocBm = new Dictionary<AocStep,IEnumerable<AocStep>>()
    {
        //{new AocStep("BOP","I"), Enumerable.Empty<AocStep>()},
        {new AocStep("MC","I"), new AocStep[]{new AocStep("BOP","I")}},
        {new AocStep("CF","I"), new AocStep[]{new AocStep("BOP","I"), new AocStep("MC","I")}},
        {new AocStep("IA","I"), new AocStep[]{new AocStep("BOP","I"), new AocStep("MC","I"), new AocStep("CF","I")}},
        {new AocStep("YCU","I"), new AocStep[]{new AocStep("BOP","I"), new AocStep("MC","I"), new AocStep("CF","I"), new AocStep("IA","I")}},
        
        {new AocStep("CL","C"), new AocStep[]{new AocStep("BOP","I"), new AocStep("MC","I"), new AocStep("CF","I"), new AocStep("IA","I"), new AocStep("YCU","I"), 
                                            }},
    
        {new AocStep("EA","C"), new AocStep[]{new AocStep("BOP","I"), new AocStep("MC","I"),new AocStep("CF","I"), new AocStep("IA","I"), new AocStep("YCU","I"),
                                              new AocStep("CL","C"),}},
        {new AocStep("AM","C"), new AocStep[]{new AocStep("BOP","I"), new AocStep("MC","I"),new AocStep("CF","I"), new AocStep("IA","I"), new AocStep("YCU","I"),
                                              new AocStep("CL","C"), new AocStep("EA","C"),}},
        {new AocStep("EOP","C"), new AocStep[]{new AocStep("BOP","I"), new AocStep("MC","I"),new AocStep("CF","I"), new AocStep("IA","I"), new AocStep("YCU","I"),
                                               new AocStep("CL","C"), new AocStep("EA","C"), new AocStep("AM","C"),}},
        //{new AocStep("CF","C"), Enumerable.Empty<AocStep>()},
    };

In [1]:
await CheckAocStepStructureAsync(inputRawVariables, parentBm, referenceBm, fullAocBm)

## Actual

In [1]:
var groupOfContract = "DT1.1";

In [1]:
var inputIfrsVariables = new IfrsVariable[]{
    new IfrsVariable{Partition = partition.Id, Value = 1.0, DataNode = groupOfContract, AocType = "BOP", Novelty = "C", AccidentYear = null, AmountType = "PR", EstimateType = "AA"},
    new IfrsVariable{Partition = partition.Id, Value = 1.0, DataNode = groupOfContract, AocType = "CF", Novelty = "C", AccidentYear = null, AmountType = "PR", EstimateType = "AA"},
    new IfrsVariable{Partition = partition.Id, Value = 1.0, DataNode = groupOfContract, AocType = "CF", Novelty = "C", AccidentYear = null, AmountType = "ACA", EstimateType = "A"},
    new IfrsVariable{Partition = partition.Id, Value = 1.0, DataNode = groupOfContract, AocType = "WO", Novelty = "C", AccidentYear = null, AmountType = "PR", EstimateType = "OA"},
};

In [1]:
parentBm = null;

In [1]:
referenceBm = null;

In [1]:
fullAocBm = new Dictionary<AocStep,IEnumerable<AocStep>>()
    {
        //{new AocStep("BOP","I"), Enumerable.Empty<AocStep>()},
        {new AocStep("CF","C"), new AocStep[]{new AocStep("BOP","I"),}},
        {new AocStep("WO","C"), new AocStep[]{new AocStep("BOP","I"), new AocStep("CF","C"),}},
        {new AocStep("AM","C"), new AocStep[]{new AocStep("BOP","I"), new AocStep("CF","C"), new AocStep("WO","C"),}},
        {new AocStep("EOP","C"), new AocStep[]{new AocStep("BOP","I"), new AocStep("CF","C"), new AocStep("WO","C"), new AocStep("AM","C"),}},
    };

In [1]:
await CheckAocStepStructureAsync(inputIfrsVariables, parentBm, referenceBm, fullAocBm)